In [ ]:
# Importando as bibliotecas

In [382]:
import requests as r
import pandas as pd
from datetime import datetime
import numpy as np
from pandasql import sqldf
import pyodbc as odc


# Requisitando os dados na API

In [430]:
url = 'https://covid19-brazil-api.now.sh/api/report/v1'
df = r.request('GET',url)

# Convertendo o Json em um DataFrame

In [456]:
def JsonDataframe():
    #Criando um DataFrame vazio.
    df2 = pd.DataFrame()
    i = 0
    
    #Obetendo os dados da API
    df = r.request('GET',url)
    
    #Obtendo a data atual do sistema.
    data_atual = datetime.today().strftime('%Y-%m-%d')

    #Lendo linha a linha do dicionaro 'Data' vindo do arquivo Json.
    #O dicionario e composto por chave e valor.

    for i in range(len(df.json()['data'])):
        df1 = pd.DataFrame(df.json()['data'][i], index=[0])

        #Alterando o formato da data
        df1['datetime'] = pd.to_datetime(df1['datetime']).dt.strftime("%Y-%m-%d")

        #Adicionando os registros ao dataframe = df2
        df2 = df2.append(df1)


In [458]:
df2['datetime'].unique()

array(['2022-04-22'], dtype=object)

# Fazer Select no Banco para conferir valores repetidos.

In [473]:
def SelectBanco ():
    
    con_string = ('DRIVER={SQL Server};'
              'SERVER=DESKTOP-AF7BA3M;'
              'DATABASE=CovidDB;')

    cnxn = odc.connect(con_string)
    query = """SELECT * FROM Tb_Covid1"""
    #Passando o retorno da consulta para um dataframe = 'result'
    result = pd.read_sql(query, cnxn)
    return result

# Função para verificar a ultima data dos registros.

In [466]:
def IncrementoBanco (query,data_atual,df2):
    df0 = pd.DataFrame(query)
    if (df0['datetime'].max() < data_atual) & (df2['datetime'].max() > df0['datetime'].max()) :
            GravarNoBanco(df2)   

# Insert - SQL

In [265]:
def GravarNoBanco(df):

        cnxn = odc.connect('DRIVER={SQL Server};'
                       'SERVER=DESKTOP-AF7BA3M;'
                        'DATABASE=CovidDB;')
        cursor = cnxn.cursor()

        for index , linha in df.iterrows():
            cursor.execute("insert into Tb_Covid1(uid, uf, state,cases,deaths,suspects,refuses,datetime) values (?,?,?,?,?,?,?,?)",linha.uid,linha.uf,linha.state,linha.cases,linha.deaths,linha.suspects,linha.refuses,linha.datetime)
            cursor.commit()

# Instancioando as funçoes.

In [475]:
#Obtendo os dados da API
JsonDataframe()

# Query no banco de dados verificando a ultima data.
query = SelectBanco()

#Incrementando os dados no banco
IncrementoBanco(query,data_atual,df2)

# Consultando o banco para verificar os valores inseridos.

In [476]:
query['datetime'].max()

'2022-04-22'

# Agrupando e somando os valores por estado.

In [453]:
url2 = 'https://servicodados.ibge.gov.br/api/v1/localidades/estados'
df1 = r.request('GET',url2)

